In [109]:
import pandas as pd
import pandas_datareader as pdr
import FinanceDataReader as fdr
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from tabulate import tabulate
from pykrx import stock

In [110]:
def getData(stocks, start, end):
    returns = pd.DataFrame()
    for i in stocks:
        df = pd.DataFrame(stock.get_market_ohlcv_by_date(start, end, i)['종가']).rename(
            columns = {'종가': i}).pct_change()
        returns = pd.concat([returns, df], axis = 1)
        
    meanReturn = returns.mean()
    covMatrix = returns.cov()
    returns = returns.dropna()
    return returns, meanReturn, covMatrix

def portfolioPerformance(weights, meanReturn, covMatrix, Days):
    port_returns = np.sum(meanReturn*weights)*Days
    std = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(Days)
    return port_returns, std


def historicalVaR(returns, alpha=5):
    """
    Read in a pandas dataframe of returns / a pandas series of returns
    Output the percentile of the distribution at the given alpha confidence level
    """
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)

    # A passed user-defined-function will be passed a Series for evaluation.
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalVaR, alpha=alpha)

    else:
        raise TypeError("Expected returns to be dataframe or series")
        
def historicalCVaR(returns, alpha=5):
    """
    Read in a pandas dataframe of returns / a pandas series of returns
    Output the CVaR for dataframe / series
    """
    if isinstance(returns, pd.Series):
        belowVaR = returns <= historicalVaR(returns, alpha=alpha)
        return returns[belowVaR].mean()

    # A passed user-defined-function will be passed a Series for evaluation.
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalCVaR, alpha=alpha)

    else:
        raise TypeError("Expected returns to be dataframe or series")

In [113]:
tickers = stock.get_market_ticker_list(market="KOSPI")[:10]
returns, meanReturn, covMatrix = getData(tickers, start, end)

weights = np.random.random(len(returns.columns))
weights /= np.sum(weights)

returns['portfolio'] = returns.dot(weights)
returns

,095570,006840,027410,282330,138930,001460,001465,001040,079160,00104K,portfolio
날짜,,,,,,,,,,,
2020-01-03,0.009901,0.002869,0.010909,0.002967,0.002677,0.000000,0.000000,-0.012793,-0.011633,-0.011331,-0.001354
2020-01-06,-0.013725,-0.010014,-0.034173,-0.014793,-0.018692,-0.012685,-0.004032,-0.024838,-0.032360,-0.025788,-0.020270
2020-01-07,0.000000,0.020231,0.024209,0.018018,0.010884,0.000000,-0.004049,0.008859,0.018229,0.008824,0.011782
2020-01-08,-0.018887,-0.043909,-0.038182,0.000000,-0.013459,-0.006424,-0.008130,-0.002195,-0.043275,-0.008746,-0.014797
2020-01-09,0.005066,0.038519,0.018904,-0.017699,-0.020464,0.004310,-0.004098,-0.005501,0.014034,-0.010294,-0.003952
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,-0.032538,-0.038697,0.025370,-0.007273,0.001724,0.001799,0.003584,0.048603,-0.008584,0.048666,0.008270
2020-12-24,0.020179,0.067797,0.030928,0.014652,0.022375,0.019749,-0.003571,0.024334,0.030303,0.028443,0.025965
2020-12-28,0.038462,-0.011905,0.016000,-0.003610,0.018519,-0.001761,-0.021505,0.010181,-0.004202,0.011645,0.010128


In [104]:
historicalVaR(returns['portfolio'])

-0.03146755148026095

In [105]:
historicalCVaR(returns['portfolio'])

-0.04626397804546535

In [120]:
Days = 10

VaR = -historicalVaR(returns['portfolio'])*np.sqrt(Days)
CVaR = -historicalCVaR(returns['portfolio'])*np.sqrt(Days)
pRet, pStd = portfolioPerformance(weights, meanReturn, covMatrix, Days)

InitialInvestment = 10000000
pRet

-0.001469484887505663

In [119]:
print('Expected Portfolio Return:'    , round(InitialInvestment*pRet, 2))
print('Value at Risk 95th CI:   :'    , round(InitialInvestment*VaR, 2))
print('Conditional VaR 95th CI  :'    , round(InitialInvestment*CVaR, 2))

Expected Portfolio Return: -14694.85
Value at Risk 95th CI:   : 1174113.12
Conditional VaR 95th CI  : 1754267.05
